In [ ]:
import os

import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import numpy as np
import pandas as pd
import pyteomics.mztab
import seaborn as sns

In [ ]:
# Plot styling.
plt.style.use(['seaborn-white', 'seaborn-talk'])
plt.rc('font', family='sans-serif')
sns.set_palette('Set1')
sns.set_context('talk')

In [ ]:
classes = pd.read_csv('filenames.csv', names=['filename', 'class'])
classes['filename'] = (classes['filename'].str.split('.')
                       .apply(lambda x: x[0]))
classes = classes.set_index('filename')

In [ ]:
psms = []
mztab_dir = '../data/processed/ann_solo'
for filename in os.listdir(mztab_dir):
    file_key = filename.split('.')[0]
    if file_key in classes.index:
        psms.append(pyteomics.mztab.MzTab(os.path.join(mztab_dir, filename))
                    .spectrum_match_table[['sequence', 'charge',
                                           'exp_mass_to_charge',
                                           'calc_mass_to_charge',
                                           'search_engine_score[1]']])
psms = pd.concat(psms, ignore_index=True)

In [ ]:
def get_mass_groups(ssms, tol_mass, tol_mode, min_group_size):
    psms_remaining = ssms.sort_values('search_engine_score[1]',
                                      ascending=False)
    psms_remaining['mass_diff'] = ((psms_remaining['exp_mass_to_charge'] -
                                    psms_remaining['calc_mass_to_charge']) *
                                   psms_remaining['charge'])

    # Start with the highest ranked PSM.
    mass_groups = []
    while psms_remaining.size > 0:
        # Find all remaining PSMs within the mass difference window.
        mass_diff = psms_remaining['mass_diff'].iloc[0]
        if (tol_mass is None or tol_mode not in ('Da', 'ppm') or
                min_group_size is None):
            mask = np.full(len(psms_remaining), True, dtype=bool)
        elif tol_mode == 'Da':
            mask = (np.fabs(psms_remaining['mass_diff'] - mass_diff) <=
                    tol_mass)
        elif tol_mode == 'ppm':
            mask = (np.fabs(psms_remaining['mass_diff'] - mass_diffs) /
                    psms_remaining['exp_mass_to_charge'] * 10 ** 6
                    <= tol_mass)
        mass_groups.append(psms_remaining[mask])
        # Exclude the selected PSMs from further selections.
        psms_remaining = psms_remaining[~mask]

    mass_group_stats = []
    for mass_group in mass_groups:
        mass_group_stats.append((mass_group['mass_diff'].median(),
                                 mass_group['mass_diff'].mean(),
                                 len(mass_group)))
    mass_group_stats = pd.DataFrame.from_records(
        mass_group_stats, columns=['mass_diff_median', 'mass_diff_mean',
                                   'num_psms'])
    return mass_group_stats

In [ ]:
tol_mass = 0.1
tol_mode = 'Da'
min_group_size = 20
mass_groups = get_mass_groups(psms, tol_mass, tol_mode, min_group_size)

In [ ]:
mass_groups.sort_values('mass_diff_median').to_csv(
    'modifications.csv', index=False)

In [ ]:
print(f'Number of PSMs: {len(psms):,}')
num_modified = (mass_groups[mass_groups['mass_diff_mean'].abs() > tol_mass]
                ['num_psms'].sum())
print(f'Number of modified PSMs: {num_modified:,} '
      f'({num_modified / len(psms):.1%})')

In [ ]:
mass_groups.sort_values('num_psms', ascending=False).head(20)

In [ ]:
width = 7
height = width / 1.618
fig, ax = plt.subplots(figsize=(width * 1.5, height / 1.5))

# Exclude unmodified SSMs.
mask = mass_groups['mass_diff_median'].abs() > tol_mass
ax.bar(mass_groups[mask]['mass_diff_median'], mass_groups[mask]['num_psms'],
       width=0.4, color='black')

# Annotate the most frequent modifications.
modifications = [('first isotopic peak', 0, 150000),
                 (None, 0, 0),
                 ('oxidation', 0, 80000),
                 (None, 0, 0),
                 (None, 0, 0),
                 ('dioxidation', 0, 40000),
                 ('amidation', -15, 80000),
                 (None, 0, 0),
                 (None, 0, 0),
                 (None, 0, 0),
                 (None, 0, 0),
                 (None, 0, 0),
                 ('Asn → Trp\nsubstitution', 0, 40000),
                 (None, 0, 0),
                 (None, 0, 0),
                 (None, 0, 0),
                 ('3 protons → iron', 0, 120000)]
for (annot, x, y), mass_group in zip(
        modifications, mass_groups.sort_values(
            'num_psms', ascending=False).head(20)[1:].itertuples()):
    if annot is not None:
        ax.annotate(
            annot,
            (mass_group.mass_diff_median, mass_group.num_psms + 5000),
            (mass_group.mass_diff_median + x, y),
            arrowprops={'arrowstyle': '<-', 'linewidth': 1}, ha='center')

ax.set_xlim((-50, 100))

# Format y-axis numbers.
ax.yaxis.set_major_formatter(mticker.StrMethodFormatter('{x:,g}'))

sns.despine(ax=ax)

ax.set_xlabel('Precursor mass difference (Da)')
ax.set_ylabel(f'Number of PSMs')

plt.savefig('mass_diff.png', dpi=300, bbox_inches='tight')
plt.show()
plt.close()